In [ ]:
# Question
#
# This task requires you to create a classifier for horses or humans using
# the provided data. Please make sure your final layer is a 1 neuron, activated by sigmoid as shown.
# Please note that the test will use images that are 300x300 with 3 bytes color depth so be sure to design your neural network accordingly

# =================================================== #
# =================================================== #
# ====== 모델링 할 때는 포함, 시험 볼때는 제거 ====== #
# 텐서플로우 버전 충돌로 인하여 
# 반드시 tensorflow 버전을 2.1.0 버전 설치 후 
# 모델작업 진행 해주셔야 합니다.
# 시험보실 때는 아래 
# !pip install tensorflow==2.1.0 명령어 필요 없습니다
# 텐서플로우 2.1.0 버전 재설치 코드
!pip install tensorflow==2.1.0

# ====== 모델링 할 때는 포함, 시험 볼때는 제거 ====== #
# =================================================== #

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - Horses Or Humans (Type B)
# val_loss: 0.51 (더 낮아도 안 좋고, 높아도 안 좋음!)
# val_acc: 관계없음
# =================================================== #
# =================================================== #



import tensorflow as tf
import urllib
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

def solution_model():
    _TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
    _TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
    urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
    local_zip = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/horse-or-human/')
    zip_ref.close()
    urllib.request.urlretrieve(_TEST_URL, 'validation-horse-or-human.zip')
    local_zip = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/validation-horse-or-human/')
    zip_ref.close()

    train_datagen = ImageDataGenerator(
        #Your code here. Should at least have a rescale. Other parameters can help with overfitting.)
        rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest', 
        )
    validation_datagen = ImageDataGenerator(
        #Your Code here)
        rescale=1. / 255,
        )
    
    TRAINING_DIR = "tmp/horse-or-human/"
    VALID_DIR = 'tmp/validation-horse-or-human/'
    train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                              batch_size=32, 
                                                              target_size=(300, 300), 
                                                              class_mode='binary', 
                                                            )
        #Your Code Here)

    validation_generator = validation_datagen.flow_from_directory(VALID_DIR, 
                                                              batch_size=32, 
                                                              target_size=(300, 300), 
                                                              class_mode='binary',
                                                            )
        #Your Code Here)


    model = tf.keras.models.Sequential([
        # Note the input shape specified on your first layer must be (300,300,3)
        # Your Code here
        Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        MaxPooling2D(2, 2), 
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Flatten(), 
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        
        # This is the last layer. You should not change this code.
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.summary()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc'],
                  )

    checkpoint_path = "tmp_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss', 
                                verbose=1)

    model.fit(train_generator, 
                        validation_data=(validation_generator),
                        epochs=10,
                        callbacks=[checkpoint],
                        )

    model.load_weights(checkpoint_path)

    # NOTE: If training is taking a very long time, you should consider setting the batch size appropriately on the generator, and the steps per epoch in the model.fit#
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-horses-or-humans-type-B.h5")

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 21.3MB/s 
     |████████████████████████████████| 450kB 39.0MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=7e6ad3241a61c7ec570c48fd926df00e6bcafb104d6b0436e6e9d94cd2d04b45
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens